In [2]:
!pip install soynlp

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
from pathlib import Path
import pandas as pd
import time
import os
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [5]:
path = Path('/content/gdrive/MyDrive/chat_bot/ReviewReactionBot')
%cd /content/gdrive/MyDrive/chat_bot/ReviewReactionBot
!ls

/content/gdrive/MyDrive/chat_bot/ReviewReactionBot
checkpoints	preprocessing	  requirements.txt	     train_koGPT2.py
data		preprocessing.py  Seq2Seq_model.py
lightning_logs	__pycache__	  Seq2Seq_Transformer.ipynb
model_chp	README.md	  small_transformer.png


In [6]:
# Custom function import # 구글드라이브 경로에 위치
from Seq2Seq_model import transformer, CustomSchedule, loss_function
from preprocessing import pre_processing_text

In [7]:
df = pd.read_csv(path / 'data/ChatbotData.csv')
print(df.shape)
df.head()

(565370, 3)


,Q,A,star_avg
0,와 진짜 대박 맛있어서 암청 많이 먹다가 리뷰하는거 까먹어서 지금 남기네요 ㄷㄷㄷ ...,맛있게 드셨다니 저희도 정말 기쁩니다 ㅎㅎ 더 맛있고 정성스럽게 조리할테니 치킨 생...,5.0
1,치킨이 양이 적어보이는데 치킨 덩어리가 엄청커요!! 목덕후인 저는 목이 없어 슬프지...,소중한 사진 리뷰 감사합니다 ! 앞으로도 더 맛있는 음식으로 찾아뵐게요 ~~ 좋은 ...,4.0
2,처음시켜 보는데 상담히 만족합니다. 배달 주문하기 힘드신 어머님이 부탁하셔서 시켜드...,첫 주문에 만족하셨다니 정말 다행이에요 ㅎㅎ 앞으로도 초심 잃지 않고 저희 모두 열...,5.0
3,맛있게 잘 먹었습니다,소중한 리뷰 감사합니다 ! 더 맛있고 정성스러운 음식으로 찾아뵐게요 좋은 하루 보내...,5.0
4,맛있었어요. 감사합니다.,맛있게 드셨다니 정말 다행이에요 감사합니다 ! 좋은 하루 보내시고 항상 건강하세요><,5.0


## Tokenized with tensorflow SubwordTextEncoder

In [8]:
text_list = [text for text in df.Q.to_list() + df.A.to_list() if text] # if None False
len(text_list)

1130740

In [9]:
%%time
# 서브워드텍스트인코더를 사용하여 질문, 답변 데이터로부터 단어 집합(Vocabulary) 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    text_list, target_vocab_size=32000)

# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

CPU times: user 9min 43s, sys: 3.88 s, total: 9min 47s
Wall time: 9min 47s


In [10]:
# tokenizer 작동 확인
print(f'Vocab size:{tokenizer.vocab_size}')
# 임으의 문장 encodign test
print(tokenizer.encode(text_list[20]))
print(tokenizer.encode('이 집 짬뽕 맛있네요.'))
for ts in tokenizer.encode('이 집 짬뽕 맛있네요.'):
  print(f'{ts} -- {tokenizer.decode([ts])}')

Vocab size:31893
[201, 194, 5199, 2959, 935, 2174, 421, 478, 2865, 849, 3617, 201, 2174, 105, 26, 101, 24]
[75, 966, 12652, 267, 31683]
75 -- 이 
966 -- 집 
12652 -- 짬뽕 
267 -- 맛있네요
31683 -- .


In [11]:
# train data, sentence pair Tokenizing

MAX_LENGTH = 40 # 임의로 설정한 max token

def tokenize_and_filter(inputs, outputs):
  tokenize_inputs, tokenize_outputs = [], []

  for (sent1, sent2) in zip(inputs, outputs):

    sent1 = START_TOKEN + tokenizer.encode(sent1) + END_TOKEN
    sent2 = START_TOKEN + tokenizer.encode(sent2) + END_TOKEN

    tokenize_inputs.append(sent1)
    tokenize_outputs.append(sent2)

  # padding
  tokenize_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenize_inputs, maxlen=MAX_LENGTH, padding='post'
  )
  tokenize_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenize_outputs, maxlen=MAX_LENGTH, padding='post'
  )

  return tokenize_inputs, tokenize_outputs

review_encode, answer_encode = tokenize_and_filter( df['Q'], df['A'] )

# 최종 학습 데이터 pair
print(review_encode.shape, answer_encode.shape)

(565370, 40) (565370, 40)


## Training Transformer model

### Data batch 구성

In [12]:
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': review_encode,
        'dec_inputs': answer_encode[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {   # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다. # 교사 강요을 위한 output(정답)
        'outputs': answer_encode[:, 1:] 
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [13]:
tf.keras.backend.clear_session()

# Hyper-parameters
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)


learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

# check-points
checkpoint_path = "/content/gdrive/MyDrive/chat_bot/ReviewReactionBot/checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
    # period= # 만약 1 epoch씩 저장하지 않으면
    )

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))



(1, 31895, 256)
(1, 31895, 256)


In [14]:
EPOCHS = 10 # 1 epochs 50분,,
model.fit(dataset, 
          epochs=EPOCHS,
          callbacks=[cp_callback])


Epoch 1/10
8834/8834 [==============================] - 2307s 260ms/step - loss: 1.7777 - accuracy: 0.4593

Epoch 00001: saving model to /content/gdrive/MyDrive/chat_bot/ReviewReactionBot/checkpoints/cp-0001.ckpt
Epoch 2/10
8834/8834 [==============================] - 2297s 260ms/step - loss: 1.1244 - accuracy: 0.5301

Epoch 00002: saving model to /content/gdrive/MyDrive/chat_bot/ReviewReactionBot/checkpoints/cp-0002.ckpt
Epoch 3/10
8834/8834 [==============================] - 2298s 260ms/step - loss: 1.0256 - accuracy: 0.5447

Epoch 00003: saving model to /content/gdrive/MyDrive/chat_bot/ReviewReactionBot/checkpoints/cp-0003.ckpt
Epoch 4/10
8834/8834 [==============================] - 2291s 259ms/step - loss: 0.9732 - accuracy: 0.5534

Epoch 00004: saving model to /content/gdrive/MyDrive/chat_bot/ReviewReactionBot/checkpoints/cp-0004.ckpt
Epoch 5/10
8834/8834 [==============================] - 2306s 261ms/step - loss: 0.9381 - accuracy: 0.5595

Epoch 00005: saving model to /content/gd

## 최종 모델 Checkpoint load


In [15]:
# 최종 checkpoint load
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# 새로운 model 객체
model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)


model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
model.load_weights(latest)

/content/gdrive/MyDrive/chat_bot/ReviewReactionBot/checkpoints/cp-0010.ckpt
(1, 31895, 256)
(1, 31895, 256)


In [16]:
def evaluate(sentence):
  sentence = pre_processing_text(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

output = predict('이 집 짬뽕 진짜 맛있네요!')

Input: 이 집 짬뽕 진짜 맛있네요!
Output: 우앙~~~ 넘넘 감사드립니당 이쁜 리뷰까징~~~ 더 맛있는 스시데이가 되도록 노력하겠습니당!! 늘 아낌없는 사랑을 주시는 울 회원님들께 진심으로 감사드립니당!!


In [17]:
new_sentence = ['이 집 짱뽕 진짜 맛있네요',
                '음식이 다 식어서 왔어여,,, ㅠㅠ',
                '공짜로 줘도 다시는 안먹는다ㅎㅎㅎㅎㅎ',
                '공짜로 줘도 다시는 안먹는다.',
                'ㄷㄷㄷㄷ']

In [20]:
for text in new_sentence:
  predict(text)
  print('\n')

Input: 이 집 짱뽕 진짜 맛있네요
Output: 오랜만에 찾아주시고 맛있게 드셔주셔서 너무너무 감사합니다 항상 쭈블링 믿고 찾아주셔서 정말 감사합니다~ 앞으로도 한결같은 마음으로 변함없는 맛ㆍ질ㆍ양으로 정성 듬뿍담아 보답하겠습니다!!! 오늘도 즐거운 하루 되세요~


Input: 음식이 다 식어서 왔어여,,, ㅠㅠ
Output: 소중한 리뷰 남겨주셔서 감사합니다. 다음에는 만족스러운 식사를 하실 수 있도록 노력하겠습니다. 감사합니다


Input: 공짜로 줘도 다시는 안먹는다ㅎㅎㅎㅎㅎ
Output: 넘넘 감사합니당 더 정성과 정직한 재료로 회원님의 입맛에 100 만족하는 단골 맛집이 되도록 노력하겠습니당 회원님의 이쁜 리뷰 진심으로 감사드리며 울 회원님 오늘도 행복한 하루 보내시라고 김뿌라 가족들이 응원합니당!! 화이팅!!


Input: 공짜로 줘도 다시는 안먹는다.
Output: 에고공,,, 입맛에 맞지 않으셨군요.. 좀 더 노력하는 김뿌라가 되겠습니다. 오늘도 행복한 하루 보내세요~


Input: ㄷㄷㄷㄷ
Output: 소중한 리뷰 주셔서 고맙습니다. 저희는 언제나 한그릇 한그릇에 정성을 담아 고객님들의 건강한 입맛을 위해 최선을 다하여 준비하여 드리겠습니다. 즐거움이 가득한 하루 보내십시요 ...


